In [1]:
%pip install -U semantic-link

StatementMeta(, 38ea1c9e-31e9-4caf-b402-b5dd4b67bd31, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 146.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.7/341.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 138.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#import necessary libraries
import sempy.fabric as fabric
import pandas as pd
from datetime import date,datetime


###########################################################################################################
### Set Parameters
###########################################################################################################
#Define lakehouse and schema names
workspace_name = "<your workspace name>" #this is the workspace name where the notebook runs, and where the lakehouse lives
lakehouse_name = "<your lakehouse name>" #this is the name of the lakehouse, in the event that it needs to becreated by this notebook
schema_name = "<your schema name>"  #This is the name of the schema.  If you want a schema-enabled lakehouse, you MUST create the lakehouse ahead of time, by hand


StatementMeta(, 38ea1c9e-31e9-4caf-b402-b5dd4b67bd31, 9, Finished, Available, Finished)

In [3]:
#set default values
lakehouse_description = "Fabric Inventory Tool Lakehouse"  # this is only used if the notebook creates a new lakehouse
full_lakehouse_name = f'{workspace_name}.{lakehouse_name}.{schema_name}'

StatementMeta(, 38ea1c9e-31e9-4caf-b402-b5dd4b67bd31, 10, Finished, Available, Finished)

In [8]:
# create tables
#create the log table. 
logquery = f'CREATE TABLE IF NOT EXISTS {full_lakehouse_name}.inventorylog (timeval TIMESTAMP, category STRING, Message STRING) USING DELTA'
#print(logquery)
spark.sql(logquery)
print("Log Table Created")
workspacequery = f'CREATE TABLE IF NOT EXISTS {full_lakehouse_name}.workspaces (timeval DATE, workspace_name STRING,workspace_id STRING,state STRING, domain_id STRING, capacity_id STRING,type STRING)'
spark.sql(workspacequery)
print("Workspace Table created")
itemsquery = f"""
    CREATE TABLE IF NOT EXISTS 
    {full_lakehouse_name}.fabricitems (timeval DATE, item_name STRING, item_id STRING,Description STRING, item_type STRING, workspace_id STRING,capacity_id STRING,principal_type STRING, principal_name STRING,last_updated STRING,state STRING)
    """
spark.sql(itemsquery)
print("Items table created")

capacityquery = f'CREATE TABLE IF NOT EXISTS {full_lakehouse_name}.capacities (timeval DATE, capacity_name STRING, capacity_id STRING,sku STRING,region STRING,state STRING)'
spark.sql(capacityquery)
print("Capacities table created")

StatementMeta(, 38ea1c9e-31e9-4caf-b402-b5dd4b67bd31, 15, Finished, Available, Finished)

Log Table Created
Workspace Table created
Items table created
Capacities table created


In [5]:

# check if lakehouse exists.  If not, create a lakehouse 
try:
    lakehouse_object = mssparkutils.lakehouse.get(lakehouse_name)
    lakehouse_id=lakehouse_object.id
    print("Lakehouse exists")
except Exception as e:
        print ("Lakehouse does not exist.  Attempting to create lakehouse...")
        try:
            fabric.create_lakehouse(display_name=lakehouse_name,description=lakehouse_description,workspace=workspace_name,enable_schema=True)
            lakehouse_object = mssparkutils.lakehouse.get(lakehouse_name)
            print("Lakehouse created with schema enabling")
            lakehouse_id=lakehouse_object.id
            #schema_name = "dbo"
        except Exception as e:
            print('Unable to create lakehouse')
            print(e)
    
#check if schema exists (if schema-enabled)
if schema_name == "dbo":
    print ("Default schema.  No check performed")
else:
    schema_sql  = f"CREATE SCHEMA IF NOT EXISTS {workspace_name}.{lakehouse_name}.{schema_name}"
    spark.sql(schema_sql)
    print(f"Schema {schema_name} created.")
    

StatementMeta(, 38ea1c9e-31e9-4caf-b402-b5dd4b67bd31, 12, Finished, Available, Finished)

Lakehouse exists
Schema fitschema created.
